In [1]:
#XGBOOST
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

plt.rcParams['font.family'] = 'Times New Roman'
np.set_printoptions(suppress=True,linewidth=np.inf)
import warnings
warnings.filterwarnings('ignore')

# 读取数据
data = pd.read_csv('../Loading_Results.csv')  # 修改路径为你的实际路径
input_features = [f'LP{i}-{d}' for i in range(1, 5) for d in ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']]
output_feature = 'Knuckle_S'

X = data[input_features].values
y = data[[output_feature]].values.reshape(-1, 1)



target = 0.99  # 假设你得到这个随机数（约值）
tolerance = 0.39

# ✅ 遍历不同的随机种子寻找最佳 random_state（改写核心）
for seed in np.arange(1):                                     
    
    # 非线性特征扩展
    DegreePoly=2
    DegreeSin=1
    DegreeCos=1
    def NL(x, DegreePoly, DegreeSin, DegreeCos):
        poly = PolynomialFeatures(degree=DegreePoly, include_bias=False)
        x_poly = poly.fit_transform(x) if DegreePoly > 0 else x
        x_sin = np.hstack([np.sin((i + 1)/1932 * x) for i in range(DegreeSin)]) if DegreeSin > 0 else None
        x_cos = np.hstack([np.cos((i + 1)/1932 * x) for i in range(DegreeCos)]) if DegreeCos > 0 else None
        x_final = x_poly
        if x_sin is not None:
            x_final = np.hstack([x_final, x_sin])
        if x_cos is not None:
            x_final = np.hstack([x_final, x_cos])
        return x_final

    # 特征构造与标准化
    X_feat = NL(X, DegreePoly, DegreeSin, DegreeCos)
    X_scaled=X_feat
    #scaler = StandardScaler()
    #X_scaled = scaler.fit_transform(X_feat)                                #树模型的算法通常是切片信息增益，因此可以不用标准化处理
    #sample_num=X.shape[0]
    #X_scaled1=np.hstack([np.ones((X.shape[0],1)),X_scaled])  
    
    # 划分训练与测试集        
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.15, random_state=21426)         #二阶=21426   一阶21379  

    for seed2 in np.arange(1):
    # 构建 XGBoost 模型
        XGB = XGBRegressor(
            tree_method='hist',
            n_estimators=300,           #Number of TreeModels=300
            learning_rate=0.085,         #0.085  #一阶=0.05
            max_depth=7,                #树深；7
            max_leaves=0,               #默认为0则表示，叶子节点不做限制
            min_child_weight=1,         #一个叶子节点中的Σhi，回归任务的一个样本的hi=1
            subsample=0.8,              #样本随机；二阶=0.8
            colsample_bytree=0.8,       #特征随机；二阶=0.8
            random_state=13536,         #随机种子；二阶13536    #一阶13564       
            min_split_loss=0.087,       #叶子数量惩罚，γT的γpenalty；二阶=0.087
            reg_lambda=0.62,            #叶子权值分散惩罚，λ pennalty（L2平方项）；二阶=0.62    #一阶=0.77
            reg_alpha=0.19,             #叶子权值分散惩罚，α pennalty（L1绝对值项）；二阶----0.19       #一阶=0.01
            verbosity=0
        )
        # Step 1: 计算 Weibull 分布风险权重（基于 y_train）
        from scipy.stats import weibull_min
        stress_train = y_train[:, 0]
        k = 8.2               #二阶8.2    一阶2.5
        λ = np.percentile(stress_train, 95)
        weibull_probs = weibull_min.cdf(stress_train, c=k, scale=λ)
        w_stress = (1 + 10 * weibull_probs).reshape(-1,1)  # 可调系数10体现风险敏感程度
        #w_stress = np.ones_like(stress_train).reshape(-1,1)

        XGB.fit(X_train, y_train.ravel(),sample_weight=w_stress)

        # 预测与评估
        y_pred = XGB.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        #if abs(r2 - target) < tolerance:
        print(f"(3)Seed1 found: {seed}------Seed2 found: {seed2}------  {r2}")
print(y_pred.shape)
print(y_test.shape)
print(weibull_probs)

(3)Seed1 found: 0------Seed2 found: 0------  0.8897932743692067
(18,)
(18, 1)
[0.24038783 0.28562426 0.65730698 0.24344539 0.3857486  0.13214491 0.15248449 0.29925703 0.30369792 0.16961432 0.4118101  0.13981856 0.17971199 0.39173337 0.21420188 0.13626294 0.3056018  0.18358395 0.36503493 0.32143879 0.16121719 0.22059821 0.36632347 0.1357081  0.3558117  0.17343293 0.27185901 0.14967307 0.15953481 0.21186229 0.1560561  0.70360156 0.23904534 0.15375273 0.27101343 0.16084801 0.17885361 0.14191013 0.23946872 0.26993974 0.38348084 0.15911633 0.14655857 0.25432083 0.42955587 0.14105154 0.35300671 0.15559331 0.24927767 0.58002232 0.3316258  0.70360156 0.18633714 0.16824666 0.1660219  0.13575427 0.19417329 0.13110848 0.15390546 0.2628791  0.18051594 0.23199661 0.18317651 0.24003403 0.15233285 0.15032211 0.6717917  0.31202923 0.44327634 0.17348875 0.54017726 0.29599387 0.16419408 0.16629201 0.27479349 0.27556917 0.25980597 0.14081377 0.13478732 0.21413664 0.63484653 0.30452479 0.24315983 0.301719